In [1]:
import torch
import torch.nn as nn
import numpy as np

import time
import math
from matplotlib import pyplot

# Class

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [3]:
class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src): # 예제랑 다른부분: embedding이 없으므로 encoder에 넣지 않음
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz): # 예제랑 다른부분: 더 복잡함, float을 만들기 위함인가?
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


In [4]:
# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw): # Size:[input_data-tw, 2, tw] Ex(2500, 2, 100)
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

In [5]:
def get_data():
    # construct a littel toy dataset
    time        = np.arange(0, 400, 0.1)    
    amplitude   = np.sin(time) + np.sin(time*0.05) +np.sin(time*0.12) *np.random.normal(-0.2, 0.2, len(time))

    
    from sklearn.preprocessing import MinMaxScaler
    
    #loading weather data from a file
    #from pandas import read_csv
    #series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    # looks like normalizing input values curtial for the model
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    
    sampels = 2600
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack.. 

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

In [6]:
def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target

In [7]:
def train(train_data):
    model.train() # Turn on the train mode \o/
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_last_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [8]:
def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            output = eval_model(data)            
            total_loss += criterion(output, target).item()
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) # tensor를 numpy로 변환하여 활용하는 기법인것같군
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy() -> no need to detach stuff.. 
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i

In [9]:
# predict the next n steps based on the input data 
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps):            
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    
    # I used this plot to visualize if the model pics up any long therm struccture within the data. 
    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")    
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%steps)
    pyplot.close()
        

In [10]:
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            total_loss += len(data[0])* criterion(output, targets).cpu().item()
    return total_loss / len(data_source)

# Parameters

In [11]:
torch.manual_seed(0)
np.random.seed(0)

In [12]:
input_window = 100 # number of input steps
output_window = 1 # number of prediction steps, in this model its fixed to one
batch_size = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
train_data, val_data = get_data()
model = TransAm().to(device)

In [14]:
criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

# Main

In [15]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
   
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 

| epoch   1 |    49/  249 batches | lr 0.005000 | 22.40 ms | loss 6.50953 | ppl   671.51
| epoch   1 |    98/  249 batches | lr 0.005000 | 10.77 ms | loss 0.09258 | ppl     1.10
| epoch   1 |   147/  249 batches | lr 0.005000 | 10.33 ms | loss 0.04350 | ppl     1.04
| epoch   1 |   196/  249 batches | lr 0.005000 | 10.33 ms | loss 0.01997 | ppl     1.02
| epoch   1 |   245/  249 batches | lr 0.005000 | 10.14 ms | loss 0.01519 | ppl     1.02
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  3.30s | valid loss 0.03429 | valid ppl     1.03
-----------------------------------------------------------------------------------------
| epoch   2 |    49/  249 batches | lr 0.004750 | 10.81 ms | loss 0.03966 | ppl     1.04
| epoch   2 |    98/  249 batches | lr 0.004750 | 11.19 ms | loss 0.02234 | ppl     1.02
| epoch   2 |   147/  249 batches | lr 0.004750 | 10.37 ms | loss 0.02419 | ppl     1.02
| epoch   2 |   196/  249 batche

| epoch  13 |    49/  249 batches | lr 0.002702 | 10.18 ms | loss 0.00765 | ppl     1.01
| epoch  13 |    98/  249 batches | lr 0.002702 | 10.06 ms | loss 0.00713 | ppl     1.01
| epoch  13 |   147/  249 batches | lr 0.002702 | 10.08 ms | loss 0.00744 | ppl     1.01
| epoch  13 |   196/  249 batches | lr 0.002702 | 10.70 ms | loss 0.00696 | ppl     1.01
| epoch  13 |   245/  249 batches | lr 0.002702 | 10.34 ms | loss 0.00731 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  13 | time:  2.63s | valid loss 0.00959 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  14 |    49/  249 batches | lr 0.002567 | 10.23 ms | loss 0.00746 | ppl     1.01
| epoch  14 |    98/  249 batches | lr 0.002567 |  9.96 ms | loss 0.00736 | ppl     1.01
| epoch  14 |   147/  249 batches | lr 0.002567 | 10.27 ms | loss 0.00712 | ppl     1.01
| epoch  14 |   196/  249 batche

| epoch  25 |    49/  249 batches | lr 0.001460 | 10.40 ms | loss 0.00648 | ppl     1.01
| epoch  25 |    98/  249 batches | lr 0.001460 | 10.07 ms | loss 0.00628 | ppl     1.01
| epoch  25 |   147/  249 batches | lr 0.001460 | 10.57 ms | loss 0.00618 | ppl     1.01
| epoch  25 |   196/  249 batches | lr 0.001460 | 10.25 ms | loss 0.00581 | ppl     1.01
| epoch  25 |   245/  249 batches | lr 0.001460 | 10.22 ms | loss 0.00623 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  25 | time:  2.63s | valid loss 0.00954 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  26 |    49/  249 batches | lr 0.001387 | 10.34 ms | loss 0.00650 | ppl     1.01
| epoch  26 |    98/  249 batches | lr 0.001387 | 10.41 ms | loss 0.00614 | ppl     1.01
| epoch  26 |   147/  249 batches | lr 0.001387 | 10.13 ms | loss 0.00610 | ppl     1.01
| epoch  26 |   196/  249 batche

| epoch  37 |    49/  249 batches | lr 0.000789 | 10.34 ms | loss 0.00603 | ppl     1.01
| epoch  37 |    98/  249 batches | lr 0.000789 | 10.10 ms | loss 0.00561 | ppl     1.01
| epoch  37 |   147/  249 batches | lr 0.000789 | 10.04 ms | loss 0.00577 | ppl     1.01
| epoch  37 |   196/  249 batches | lr 0.000789 | 10.17 ms | loss 0.00543 | ppl     1.01
| epoch  37 |   245/  249 batches | lr 0.000789 | 10.14 ms | loss 0.00553 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  37 | time:  2.59s | valid loss 0.00766 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  38 |    49/  249 batches | lr 0.000749 | 10.45 ms | loss 0.00603 | ppl     1.01
| epoch  38 |    98/  249 batches | lr 0.000749 | 10.13 ms | loss 0.00556 | ppl     1.01
| epoch  38 |   147/  249 batches | lr 0.000749 | 10.37 ms | loss 0.00572 | ppl     1.01
| epoch  38 |   196/  249 batche

| epoch  49 |    49/  249 batches | lr 0.000426 | 10.44 ms | loss 0.00546 | ppl     1.01
| epoch  49 |    98/  249 batches | lr 0.000426 | 10.07 ms | loss 0.00533 | ppl     1.01
| epoch  49 |   147/  249 batches | lr 0.000426 | 10.00 ms | loss 0.00531 | ppl     1.01
| epoch  49 |   196/  249 batches | lr 0.000426 |  9.93 ms | loss 0.00505 | ppl     1.01
| epoch  49 |   245/  249 batches | lr 0.000426 | 10.08 ms | loss 0.00502 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  49 | time:  2.58s | valid loss 0.00655 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  50 |    49/  249 batches | lr 0.000405 | 10.10 ms | loss 0.00546 | ppl     1.01
| epoch  50 |    98/  249 batches | lr 0.000405 |  9.92 ms | loss 0.00534 | ppl     1.01
| epoch  50 |   147/  249 batches | lr 0.000405 | 10.24 ms | loss 0.00526 | ppl     1.01
| epoch  50 |   196/  249 batche

| epoch  61 |    49/  249 batches | lr 0.000230 | 10.40 ms | loss 0.00527 | ppl     1.01
| epoch  61 |    98/  249 batches | lr 0.000230 | 10.15 ms | loss 0.00518 | ppl     1.01
| epoch  61 |   147/  249 batches | lr 0.000230 | 10.28 ms | loss 0.00510 | ppl     1.01
| epoch  61 |   196/  249 batches | lr 0.000230 | 10.26 ms | loss 0.00489 | ppl     1.00
| epoch  61 |   245/  249 batches | lr 0.000230 | 10.10 ms | loss 0.00481 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  61 | time:  2.62s | valid loss 0.00628 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  62 |    49/  249 batches | lr 0.000219 | 10.41 ms | loss 0.00524 | ppl     1.01
| epoch  62 |    98/  249 batches | lr 0.000219 | 10.01 ms | loss 0.00519 | ppl     1.01
| epoch  62 |   147/  249 batches | lr 0.000219 | 10.00 ms | loss 0.00512 | ppl     1.01
| epoch  62 |   196/  249 batche

| epoch  73 |    49/  249 batches | lr 0.000124 | 10.26 ms | loss 0.00533 | ppl     1.01
| epoch  73 |    98/  249 batches | lr 0.000124 | 10.17 ms | loss 0.00512 | ppl     1.01
| epoch  73 |   147/  249 batches | lr 0.000124 | 10.43 ms | loss 0.00519 | ppl     1.01
| epoch  73 |   196/  249 batches | lr 0.000124 |  9.98 ms | loss 0.00493 | ppl     1.00
| epoch  73 |   245/  249 batches | lr 0.000124 | 10.06 ms | loss 0.00483 | ppl     1.00
-----------------------------------------------------------------------------------------
| end of epoch  73 | time:  2.60s | valid loss 0.00679 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  74 |    49/  249 batches | lr 0.000118 | 10.43 ms | loss 0.00532 | ppl     1.01
| epoch  74 |    98/  249 batches | lr 0.000118 | 10.01 ms | loss 0.00515 | ppl     1.01
| epoch  74 |   147/  249 batches | lr 0.000118 | 10.23 ms | loss 0.00524 | ppl     1.01
| epoch  74 |   196/  249 batche

| epoch  85 |    49/  249 batches | lr 0.000067 | 10.45 ms | loss 0.00587 | ppl     1.01
| epoch  85 |    98/  249 batches | lr 0.000067 | 10.05 ms | loss 0.00577 | ppl     1.01
| epoch  85 |   147/  249 batches | lr 0.000067 | 10.05 ms | loss 0.00621 | ppl     1.01
| epoch  85 |   196/  249 batches | lr 0.000067 | 10.33 ms | loss 0.00553 | ppl     1.01
| epoch  85 |   245/  249 batches | lr 0.000067 | 10.20 ms | loss 0.00560 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  85 | time:  2.61s | valid loss 0.00632 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  86 |    49/  249 batches | lr 0.000064 | 10.29 ms | loss 0.00594 | ppl     1.01
| epoch  86 |    98/  249 batches | lr 0.000064 | 10.73 ms | loss 0.00572 | ppl     1.01
| epoch  86 |   147/  249 batches | lr 0.000064 | 10.56 ms | loss 0.00618 | ppl     1.01
| epoch  86 |   196/  249 batche

| epoch  97 |    49/  249 batches | lr 0.000036 | 10.26 ms | loss 0.00571 | ppl     1.01
| epoch  97 |    98/  249 batches | lr 0.000036 | 10.12 ms | loss 0.00545 | ppl     1.01
| epoch  97 |   147/  249 batches | lr 0.000036 | 11.25 ms | loss 0.00565 | ppl     1.01
| epoch  97 |   196/  249 batches | lr 0.000036 | 10.02 ms | loss 0.00532 | ppl     1.01
| epoch  97 |   245/  249 batches | lr 0.000036 | 10.09 ms | loss 0.00536 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  97 | time:  2.64s | valid loss 0.00539 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  98 |    49/  249 batches | lr 0.000035 | 10.59 ms | loss 0.00568 | ppl     1.01
| epoch  98 |    98/  249 batches | lr 0.000035 |  9.97 ms | loss 0.00541 | ppl     1.01
| epoch  98 |   147/  249 batches | lr 0.000035 |  9.95 ms | loss 0.00562 | ppl     1.01
| epoch  98 |   196/  249 batche